<a href="https://colab.research.google.com/github/saharnazyp/Demand-Forecasting/blob/main/Demand_Forecasting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
!pip -q install pandas openpyxl numpy lightgbm jdatetime holidays xlsxwriter


In [24]:
!pip -q install ipywidgets plotly

In [23]:
import pandas as pd
import numpy as np
import jdatetime
import holidays
from datetime import timedelta
from lightgbm import LGBMRegressor


from google.colab import output
output.enable_custom_widget_manager()

import plotly.express as px
import ipywidgets as widgets
from IPython.display import display, clear_output
import pandas as pd

In [21]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [22]:


SHEET_NAME = "/content/drive/MyDrive/data/پیش بینی -تولید.xlsx"
BRANCH_COL = "Receiving Warehouse Name"   # ✅ شعبه/واحد رستورانی
ITEM_COL   = "code goods"

df = pd.read_excel(SHEET_NAME).rename(columns={"Holiday ": "Holiday"})

# تاریخ: 25/02/09 => 2025/02/09
df["date"] = pd.to_datetime("20" + df["Date"].astype(str), format="%Y/%m/%d", errors="raise")

# تعطیلی از داده
df["is_holiday_row"] = (df["Holiday"].astype(str).str.strip() == "تعطیل")

# متادیتا برای هر شعبه-کالا
meta = (df.groupby([BRANCH_COL, ITEM_COL], sort=False)
          .agg(title=("Tittle Goods","first"),
               unit=("Base Unit","first"))
          .reset_index())

# تجمیع روزانه برای هر شعبه-کالا
daily = (df.groupby([BRANCH_COL, ITEM_COL, "date"], as_index=False, sort=False)
           .agg(y=("Value (in base unit)","sum"),
                is_holiday=("is_holiday_row","max")))

# ساخت پنل کامل روزانه (روزهای بدون سفارش = 0)
all_dates = pd.date_range(daily["date"].min(), daily["date"].max(), freq="D")
all_series = meta[[BRANCH_COL, ITEM_COL]].drop_duplicates()

panel = (all_series.assign(key=1)
         .merge(pd.DataFrame({"date": all_dates, "key": 1}), on="key")
         .drop(columns="key")
         .merge(daily, on=[BRANCH_COL, ITEM_COL, "date"], how="left"))

panel["y"] = panel["y"].fillna(0.0)
panel["is_holiday"] = panel["is_holiday"].fillna(False).astype(bool)

panel = panel.merge(meta, on=[BRANCH_COL, ITEM_COL], how="left")

# سری زمانی یکتا برای مدل
panel["series_id"] = panel[BRANCH_COL].astype(str) + " || " + panel[ITEM_COL].astype(str)

# تقویم ایران
dow_map_iran = {5:0, 6:1, 0:2, 1:3, 2:4, 3:5, 4:6}  # 0=شنبه..6=جمعه
def iran_cal(dt):
    j = jdatetime.date.fromgregorian(date=dt.date())
    return j.month, j.day, dow_map_iran[dt.dayofweek]

panel = panel.sort_values(["series_id","date"])
panel["jmonth"], panel["jdom"], panel["dow_iran"] = zip(*panel["date"].apply(iran_cal))

# ویژگی‌های سری زمانی بر اساس سری_id
g = panel.groupby("series_id", group_keys=False, sort=False)
panel["lag1"]  = g["y"].shift(1).fillna(0.0)
panel["lag7"]  = g["y"].shift(7).fillna(0.0)
panel["lag14"] = g["y"].shift(14).fillna(0.0)
panel["roll7"]  = g["y"].shift(1).rolling(7).mean().fillna(0.0)
panel["roll28"] = g["y"].shift(1).rolling(28).mean().fillna(0.0)

FEATURES = ["dow_iran","jmonth","jdom","is_holiday","lag1","lag7","lag14","roll7","roll28","series_id"]

# آموزش مدل
train = panel.copy()
train["series_id"] = train["series_id"].astype("category")

model = LGBMRegressor(
    n_estimators=400,
    learning_rate=0.08,
    num_leaves=63,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42
)
model.fit(train[FEATURES], train["y"], categorical_feature=["series_id"])

# --- Forecast 91 days آینده (بازگشتی و سریع) ---
IR_HOL = holidays.country_holidays("IR")
dow_fa = np.array(["شنبه","یکشنبه","دوشنبه","سه‌شنبه","چهارشنبه","پنجشنبه","جمعه"], dtype=object)

series_list = meta.assign(series_id=meta[BRANCH_COL].astype(str)+" || "+meta[ITEM_COL].astype(str))["series_id"].values
branches = meta[BRANCH_COL].values
goods = meta[ITEM_COL].values
titles = meta["title"].values
units  = meta["unit"].values

# آخرین 28 روز هر سری
tail = panel.groupby("series_id", sort=False).tail(28)[["series_id","y"]]
sid_to_idx = {k:i for i,k in enumerate(series_list)}

last_vals = np.zeros((len(series_list), 28), dtype=float)
counts = np.zeros(len(series_list), dtype=int)
for sid, y in zip(tail["series_id"].values, tail["y"].values):
    i = sid_to_idx[sid]
    c = counts[i]
    if c < 28:
        last_vals[i, c] = float(y)
        counts[i] += 1
for i,c in enumerate(counts):
    if c < 28:
        last_vals[i] = np.concatenate([np.zeros(28-c), last_vals[i,:c]])

last_date = panel["date"].max()
future_dates = pd.date_range(last_date + timedelta(days=1), last_date + timedelta(days=91), freq="D")

frames = []
for d in future_dates:
    j = jdatetime.date.fromgregorian(date=d.date())
    jmonth, jdom = j.month, j.day
    dow_iran = dow_map_iran[d.dayofweek]
    is_h = (d.date() in IR_HOL)

    lag1  = last_vals[:, -1]
    lag7  = last_vals[:, -7]
    lag14 = last_vals[:, -14]
    roll7  = last_vals[:, -7:].mean(axis=1)
    roll28 = last_vals.mean(axis=1)

    X = pd.DataFrame({
        "dow_iran": dow_iran,
        "jmonth": jmonth,
        "jdom": jdom,
        "is_holiday": is_h,
        "lag1": lag1,
        "lag7": lag7,
        "lag14": lag14,
        "roll7": roll7,
        "roll28": roll28,
        "series_id": pd.Categorical(series_list)
    })

    yhat = np.clip(model.predict(X), 0, None)

    frames.append(pd.DataFrame({
        BRANCH_COL: branches,
        ITEM_COL: goods,
        "title": titles,
        "unit": units,
        "series_id": series_list,
        "date": d,
        "dow_iran": dow_iran,
        "weekday_fa": dow_fa[dow_iran],
        "jmonth": jmonth,
        "jdom": jdom,
        "is_holiday": is_h,
        "yhat": yhat
    }))

    last_vals = np.roll(last_vals, -1, axis=1)
    last_vals[:, -1] = yhat

fc_daily = pd.concat(frames, ignore_index=True)
fc_daily.head()


/tmp/ipython-input-2302379280.py:34: FutureWarning:

Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`



[LightGBM] [Warning] Categorical features with more bins than the configured maximum bin number found.
[LightGBM] [Warning] For categorical features, max_bin and max_bin_by_feature may be ignored with a large number of categories.
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.135447 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3754
[LightGBM] [Info] Number of data points in the train set: 676164, number of used features: 10
[LightGBM] [Info] Start training from score 3435.711114


,Receiving Warehouse Name,code goods,title,unit,series_id,date,dow_iran,weekday_fa,jmonth,jdom,is_holiday,yhat
0,برگر فکتوری پالادیوم,100080100063,نان هات داگ ونیز کنجدی,عدد,برگر فکتوری پالادیوم || 100080100063,2026-02-10,3,سه‌شنبه,11,21,False,272.905843
1,برگر فکتوری پالادیوم,100070100016,کاهو بیوندا پاک شده بسته بندی,گرم,برگر فکتوری پالادیوم || 100070100016,2026-02-10,3,سه‌شنبه,11,21,False,0.000000
2,چیکن فکتوری بام لند,100070100030,فلفل دلمه سبز,گرم,چیکن فکتوری بام لند || 100070100030,2026-02-10,3,سه‌شنبه,11,21,False,272.905843
3,چیکن فکتوری بام لند,100070100021,بادمجان,گرم,چیکن فکتوری بام لند || 100070100021,2026-02-10,3,سه‌شنبه,11,21,False,272.905843
4,چیکن فکتوری بام لند,100070100029,فلفل دلمه رنگی,گرم,چیکن فکتوری بام لند || 100070100029,2026-02-10,3,سه‌شنبه,11,21,False,272.905843


In [ ]:
df["Receiving Warehouse Name"].dropna().unique()[:50]


array(['برگر فکتوری پالادیوم', 'چیکن فکتوری بام لند',
       'برگر فکتوری بام لند', 'لیبرو پالادیوم', 'رستوران اکسپرس پالادیوم',
       'لیبرو ایران مال', 'لفو', 'اسپئو رویال پالادیوم',
       'روماتای پالادیوم', 'چیکن فکتوری ایران مال',
       'برگر فکتوری ایران مال', 'دلیش بام لند', 'سوپر مارکت جزیره'],
      dtype=object)

In [25]:


SHEET_NAME = "/content/drive/MyDrive/data/پیش بینی -تولید.xlsx"
BRANCH_COL = "Receiving Warehouse Name"   # شعبه/واحد رستورانی
ITEM_COL   = "code goods"

df = pd.read_excel(SHEET_NAME).rename(columns={"Holiday ": "Holiday"})
df["date"] = pd.to_datetime("20" + df["Date"].astype(str), format="%Y/%m/%d", errors="raise")
df["is_holiday_row"] = (df["Holiday"].astype(str).str.strip() == "تعطیل")

meta = (df.groupby([BRANCH_COL, ITEM_COL], sort=False)
          .agg(title=("Tittle Goods","first"),
               unit=("Base Unit","first"))
          .reset_index())

daily = (df.groupby([BRANCH_COL, ITEM_COL, "date"], as_index=False, sort=False)
           .agg(y=("Value (in base unit)","sum"),
                is_holiday=("is_holiday_row","max")))

all_dates = pd.date_range(daily["date"].min(), daily["date"].max(), freq="D")
all_series = meta[[BRANCH_COL, ITEM_COL]].drop_duplicates()

panel = (all_series.assign(key=1)
         .merge(pd.DataFrame({"date": all_dates, "key": 1}), on="key")
         .drop(columns="key")
         .merge(daily, on=[BRANCH_COL, ITEM_COL, "date"], how="left"))

panel["y"] = panel["y"].fillna(0.0)
panel["is_holiday"] = panel["is_holiday"].fillna(False).astype(bool)
panel = panel.merge(meta, on=[BRANCH_COL, ITEM_COL], how="left")

panel["series_id"] = panel[BRANCH_COL].astype(str) + " || " + panel[ITEM_COL].astype(str)

dow_map_iran = {5:0, 6:1, 0:2, 1:3, 2:4, 3:5, 4:6}
def iran_cal(dt):
    j = jdatetime.date.fromgregorian(date=dt.date())
    return j.month, j.day, dow_map_iran[dt.dayofweek]

panel = panel.sort_values(["series_id","date"])
panel["jmonth"], panel["jdom"], panel["dow_iran"] = zip(*panel["date"].apply(iran_cal))

g = panel.groupby("series_id", group_keys=False, sort=False)
panel["lag1"]  = g["y"].shift(1).fillna(0.0)
panel["lag7"]  = g["y"].shift(7).fillna(0.0)
panel["lag14"] = g["y"].shift(14).fillna(0.0)
panel["roll7"]  = g["y"].shift(1).rolling(7).mean().fillna(0.0)
panel["roll28"] = g["y"].shift(1).rolling(28).mean().fillna(0.0)

FEATURES = ["dow_iran","jmonth","jdom","is_holiday","lag1","lag7","lag14","roll7","roll28","series_id"]

train = panel.copy()
train["series_id"] = train["series_id"].astype("category")

model = LGBMRegressor(
    n_estimators=400,
    learning_rate=0.08,
    num_leaves=63,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42
)
model.fit(train[FEATURES], train["y"], categorical_feature=["series_id"])

IR_HOL = holidays.country_holidays("IR")
dow_fa = np.array(["شنبه","یکشنبه","دوشنبه","سه‌شنبه","چهارشنبه","پنجشنبه","جمعه"], dtype=object)

series_list = meta.assign(series_id=meta[BRANCH_COL].astype(str)+" || "+meta[ITEM_COL].astype(str))["series_id"].values
branches = meta[BRANCH_COL].values
goods = meta[ITEM_COL].values
titles = meta["title"].values
units  = meta["unit"].values

tail = panel.groupby("series_id", sort=False).tail(28)[["series_id","y"]]
sid_to_idx = {k:i for i,k in enumerate(series_list)}

last_vals = np.zeros((len(series_list), 28), dtype=float)
counts = np.zeros(len(series_list), dtype=int)
for sid, y in zip(tail["series_id"].values, tail["y"].values):
    i = sid_to_idx[sid]
    c = counts[i]
    if c < 28:
        last_vals[i, c] = float(y)
        counts[i] += 1
for i,c in enumerate(counts):
    if c < 28:
        last_vals[i] = np.concatenate([np.zeros(28-c), last_vals[i,:c]])

last_date = panel["date"].max()
future_dates = pd.date_range(last_date + timedelta(days=1), last_date + timedelta(days=91), freq="D")

frames = []
for d in future_dates:
    j = jdatetime.date.fromgregorian(date=d.date())
    jmonth, jdom = j.month, j.day
    dow_iran = dow_map_iran[d.dayofweek]
    is_h = (d.date() in IR_HOL)

    lag1  = last_vals[:, -1]
    lag7  = last_vals[:, -7]
    lag14 = last_vals[:, -14]
    roll7  = last_vals[:, -7:].mean(axis=1)
    roll28 = last_vals.mean(axis=1)

    X = pd.DataFrame({
        "dow_iran": dow_iran,
        "jmonth": jmonth,
        "jdom": jdom,
        "is_holiday": is_h,
        "lag1": lag1,
        "lag7": lag7,
        "lag14": lag14,
        "roll7": roll7,
        "roll28": roll28,
        "series_id": pd.Categorical(series_list)
    })

    yhat = np.clip(model.predict(X), 0, None)

    frames.append(pd.DataFrame({
        BRANCH_COL: branches,
        ITEM_COL: goods,
        "title": titles,
        "unit": units,
        "series_id": series_list,
        "date": d,
        "dow_iran": dow_iran,
        "weekday_fa": dow_fa[dow_iran],
        "jmonth": jmonth,
        "jdom": jdom,
        "is_holiday": is_h,
        "yhat": yhat
    }))

    last_vals = np.roll(last_vals, -1, axis=1)
    last_vals[:, -1] = yhat

fc_daily = pd.concat(frames, ignore_index=True)

# weekly (شنبه تا جمعه)
offset_to_saturday = (fc_daily["date"].dt.dayofweek - 5) % 7
fc_daily["week_start"] = fc_daily["date"] - pd.to_timedelta(offset_to_saturday, unit="D")

fc_weekly = (fc_daily.groupby([BRANCH_COL, ITEM_COL, "title", "unit", "week_start"], as_index=False)
                    .agg(week_total=("yhat","sum"))
                    .sort_values([BRANCH_COL, ITEM_COL, "week_start"]))

fc_daily.head(), fc_weekly.head()


/tmp/ipython-input-2704833511.py:27: FutureWarning:

Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`



[LightGBM] [Warning] Categorical features with more bins than the configured maximum bin number found.
[LightGBM] [Warning] For categorical features, max_bin and max_bin_by_feature may be ignored with a large number of categories.
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.071982 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3754
[LightGBM] [Info] Number of data points in the train set: 676164, number of used features: 10
[LightGBM] [Info] Start training from score 3435.711114


(  Receiving Warehouse Name    code goods                          title unit  \
 0     برگر فکتوری پالادیوم  100080100063         نان هات داگ ونیز کنجدی  عدد   
 1     برگر فکتوری پالادیوم  100070100016  کاهو بیوندا پاک شده بسته بندی  گرم   
 2      چیکن فکتوری بام لند  100070100030                  فلفل دلمه سبز  گرم   
 3      چیکن فکتوری بام لند  100070100021                        بادمجان  گرم   
 4      چیکن فکتوری بام لند  100070100029                 فلفل دلمه رنگی  گرم   
 
                               series_id       date  dow_iran weekday_fa  \
 0  برگر فکتوری پالادیوم || 100080100063 2026-02-10         3    سه‌شنبه   
 1  برگر فکتوری پالادیوم || 100070100016 2026-02-10         3    سه‌شنبه   
 2   چیکن فکتوری بام لند || 100070100030 2026-02-10         3    سه‌شنبه   
 3   چیکن فکتوری بام لند || 100070100021 2026-02-10         3    سه‌شنبه   
 4   چیکن فکتوری بام لند || 100070100029 2026-02-10         3    سه‌شنبه   
 
    jmonth  jdom  is_holiday        yhat week_start  


In [26]:


# اطمینان از datetime
fc_daily["date"] = pd.to_datetime(fc_daily["date"])

branches_list = sorted(fc_daily[BRANCH_COL].dropna().unique().tolist())
dd_branch = widgets.Dropdown(options=branches_list, description="شعبه:", layout=widgets.Layout(width="95%"))
dd_item = widgets.Dropdown(options=[], description="کالا:", layout=widgets.Layout(width="95%"))

# DatePicker (میلادی؛ اگر خواستی شمسیش هم می‌تونیم بسازیم)
date_picker = widgets.DatePicker(
    description='تاریخ:',
    disabled=False
)

out = widgets.Output()

def update_items(branch):
    subset = fc_daily[fc_daily[BRANCH_COL] == branch][[ITEM_COL, "title", "unit"]].drop_duplicates()
    opts = [(f"{r['title']} | کد:{r[ITEM_COL]} | واحد:{r['unit']}", r[ITEM_COL])
            for _, r in subset.sort_values(["title", ITEM_COL]).iterrows()]
    dd_item.options = opts
    if opts:
        dd_item.value = opts[0][1]

    # تنظیم بازه تاریخ‌های قابل انتخاب بر اساس پیش‌بینی‌ها
    dates = fc_daily[fc_daily[BRANCH_COL] == branch]["date"]
    if not dates.empty:
        date_picker.value = dates.min().date()
        date_picker.min = dates.min().date()
        date_picker.max = dates.max().date()

def render(branch, code_goods, picked_date=None):
    with out:
        clear_output(wait=True)
        ddf = fc_daily[(fc_daily[BRANCH_COL] == branch) & (fc_daily[ITEM_COL] == code_goods)].sort_values("date")
        if ddf.empty:
            display("داده‌ای برای این انتخاب نیست.")
            return

        title = ddf["title"].iloc[0]
        unit  = ddf["unit"].iloc[0]

        # --- نمودار هفتگی ---
        w = fc_weekly[(fc_weekly[BRANCH_COL] == branch) & (fc_weekly[ITEM_COL] == code_goods)].sort_values("week_start")
        fig = px.bar(w, x="week_start", y="week_total",
                     title=f"Weekly Forecast (Sat–Fri) - {branch} | {title} ({unit})")
        fig.update_layout(height=420, xaxis_title="Week start (Saturday)", yaxis_title=f"Weekly total ({unit})")
        fig.show()

        # --- خروجی روز انتخاب‌شده ---
        if picked_date is not None:
            day_row = ddf[ddf["date"].dt.date == picked_date]
            if not day_row.empty:
                r = day_row.iloc[0]
                display(widgets.HTML(f"""
                <div style="font-family:Tahoma; line-height:1.9; margin:10px 0; padding:10px; border:1px solid #ddd; border-radius:8px;">
                  <b>پیش‌بینی روز انتخاب‌شده:</b><br/>
                  {branch} | {title}<br/>
                  <b>تاریخ:</b> {r['weekday_fa']} {r['date'].date()}<br/>
                  <b>مقدار پیش‌بینی:</b> {r['yhat']:.2f} {unit}<br/>
                  <b>تعطیل:</b> {"بله" if r["is_holiday"] else "خیر"}
                </div>
                """))
            else:
                display("برای این تاریخ پیش‌بینی وجود ندارد.")

        # --- جدول روزانه (۹۱ روز) ---
        display(ddf[["date","weekday_fa","yhat","is_holiday","jmonth","jdom"]].reset_index(drop=True))

def on_branch(change):
    if change["name"] == "value":
        update_items(change["new"])
        if dd_item.value is not None:
            render(dd_branch.value, dd_item.value, date_picker.value)

def on_item(change):
    if change["name"] == "value":
        render(dd_branch.value, change["new"], date_picker.value)

def on_date(change):
    render(dd_branch.value, dd_item.value, change["new"])

dd_branch.observe(on_branch)
dd_item.observe(on_item)
date_picker.observe(on_date)

update_items(dd_branch.value)
display(dd_branch, dd_item, date_picker, out)
render(dd_branch.value, dd_item.value, date_picker.value)


Dropdown(description='شعبه:', layout=Layout(width='95%'), options=('اسپئو رویال پالادیوم', 'برگر فکتوری ایران …

Dropdown(description='کالا:', layout=Layout(width='95%'), options=(('آب آلبالو زرشک 750 سی سی | کد:10005030000…

DatePicker(value=datetime.date(2026, 2, 10), description='تاریخ:')

Output()

In [27]:
target_branch = "برگر فکتوری بام لند"
target_title  = "سیب زمینی"
target_date   = pd.Timestamp("2026-02-14")

ans = fc_daily[
    (fc_daily[BRANCH_COL] == target_branch) &
    (fc_daily["title"] == target_title) &
    (fc_daily["date"] == target_date)
][[BRANCH_COL, "title", "unit", "date", "weekday_fa", "yhat"]]

ans


,Receiving Warehouse Name,title,unit,date,weekday_fa,yhat


In [28]:
!pip -q install ipywidgets plotly
from google.colab import output
output.enable_custom_widget_manager()

import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import ipywidgets as widgets
from IPython.display import display, clear_output

# اطمینان از datetime
fc_daily["date"] = pd.to_datetime(fc_daily["date"])

# ---------- ساخت week_start شنبه ----------
offset_to_saturday = (fc_daily["date"].dt.dayofweek - 5) % 7
fc_daily["week_start"] = fc_daily["date"] - pd.to_timedelta(offset_to_saturday, unit="D")

# weekly summary
fc_weekly = (fc_daily.groupby([BRANCH_COL, ITEM_COL, "title", "unit", "week_start"], as_index=False)
                    .agg(week_total=("yhat","sum"))
                    .sort_values([BRANCH_COL, ITEM_COL, "week_start"]))

branches_list = sorted(fc_daily[BRANCH_COL].dropna().unique().tolist())
dd_branch = widgets.Dropdown(options=branches_list, description="شعبه:", layout=widgets.Layout(width="95%"))
dd_item   = widgets.Dropdown(options=[], description="کالا:", layout=widgets.Layout(width="95%"))
dd_week   = widgets.Dropdown(options=[], description="هفته:", layout=widgets.Layout(width="95%"))

out = widgets.Output()

def update_items(branch):
    subset = fc_daily[fc_daily[BRANCH_COL] == branch][[ITEM_COL, "title", "unit"]].drop_duplicates()
    opts = [(f"{r['title']} | کد:{r[ITEM_COL]} | واحد:{r['unit']}", r[ITEM_COL])
            for _, r in subset.sort_values(["title", ITEM_COL]).iterrows()]
    dd_item.options = opts
    if opts:
        dd_item.value = opts[0][1]

def update_weeks(branch, code_goods):
    w = fc_weekly[(fc_weekly[BRANCH_COL] == branch) & (fc_weekly[ITEM_COL] == code_goods)].sort_values("week_start")
    # گزینه‌های هفته: "شروع هفته (شنبه) تا جمعه"
    week_opts = []
    for ws in w["week_start"].unique():
        ws = pd.Timestamp(ws)
        we = ws + pd.Timedelta(days=6)
        label = f"{ws.date()}  تا  {we.date()}"
        week_opts.append((label, ws.date()))
    dd_week.options = week_opts
    if week_opts:
        dd_week.value = week_opts[0][1]

def render(branch, code_goods, week_start_date=None):
    with out:
        clear_output(wait=True)

        ddf = fc_daily[(fc_daily[BRANCH_COL] == branch) & (fc_daily[ITEM_COL] == code_goods)].copy()
        if ddf.empty:
            display("داده‌ای برای این انتخاب نیست.")
            return

        title = ddf["title"].iloc[0]
        unit  = ddf["unit"].iloc[0]

        # ----- نمودار هفتگی -----
        w = fc_weekly[(fc_weekly[BRANCH_COL] == branch) & (fc_weekly[ITEM_COL] == code_goods)].sort_values("week_start")

        fig_w = px.bar(
            w, x="week_start", y="week_total",
            title=f"Weekly Forecast (Sat–Fri) - {branch} | {title} ({unit})"
        )
        fig_w.update_layout(height=360, xaxis_title="Week start (Saturday)", yaxis_title=f"Weekly total ({unit})")
        fig_w.show()

        # ----- جزئیات هفته انتخابی -----
        if week_start_date is None and len(w) > 0:
            week_start_date = pd.to_datetime(w["week_start"].iloc[0]).date()

        ws = pd.Timestamp(week_start_date)
        we = ws + pd.Timedelta(days=6)

        week_df = ddf[(ddf["date"] >= ws) & (ddf["date"] <= we)].sort_values("date").copy()

        if week_df.empty:
            display("برای این هفته داده‌ای وجود ندارد.")
            return

        week_sum = week_df["yhat"].sum()
        week_max = week_df["yhat"].max()
        week_avg = week_df["yhat"].mean()

        display(widgets.HTML(f"""
        <div style="font-family:Tahoma; line-height:1.9; margin:8px 0; padding:10px; border:1px solid #ddd; border-radius:10px;">
          <b>جزئیات هفته:</b> {ws.date()} تا {we.date()}<br/>
          <b>جمع هفته:</b> {week_sum:,.2f} {unit} &nbsp; | &nbsp;
          <b>میانگین روزانه:</b> {week_avg:,.2f} {unit} &nbsp; | &nbsp;
          <b>بیشترین روز:</b> {week_max:,.2f} {unit}
        </div>
        """))

        # نمودار روزانه همان هفته (خطی + نقاط + علامت تعطیل)
        fig_d = go.Figure()
        fig_d.add_trace(go.Scatter(
            x=week_df["date"],
            y=week_df["yhat"],
            mode="lines+markers",
            name="Daily forecast"
        ))

        # علامت‌گذاری روزهای تعطیل
        hol = week_df[week_df["is_holiday"] == True]
        if not hol.empty:
            fig_d.add_trace(go.Scatter(
                x=hol["date"],
                y=hol["yhat"],
                mode="markers",
                name="Holiday",
                marker=dict(size=12, symbol="x")
            ))

        fig_d.update_layout(
            title=f"Daily Detail (Selected Week) - {branch} | {title} ({unit})",
            height=380,
            xaxis_title="Date",
            yaxis_title=f"Predicted daily demand ({unit})"
        )
        fig_d.show()

        # جدول شنبه تا جمعه
        display(week_df[["date","weekday_fa","yhat","is_holiday","jmonth","jdom"]].reset_index(drop=True))

def on_branch(change):
    if change["name"] == "value":
        update_items(change["new"])
        if dd_item.value is not None:
            update_weeks(dd_branch.value, dd_item.value)
            render(dd_branch.value, dd_item.value, dd_week.value)

def on_item(change):
    if change["name"] == "value":
        update_weeks(dd_branch.value, change["new"])
        render(dd_branch.value, dd_item.value, dd_week.value)

def on_week(change):
    if change["name"] == "value":
        render(dd_branch.value, dd_item.value, change["new"])

dd_branch.observe(on_branch)
dd_item.observe(on_item)
dd_week.observe(on_week)

# init
update_items(dd_branch.value)
update_weeks(dd_branch.value, dd_item.value)
display(dd_branch, dd_item, dd_week, out)
render(dd_branch.value, dd_item.value, dd_week.value)


Dropdown(description='شعبه:', layout=Layout(width='95%'), options=('اسپئو رویال پالادیوم', 'برگر فکتوری ایران …

Dropdown(description='کالا:', layout=Layout(width='95%'), options=(('آب آلبالو زرشک 750 سی سی | کد:10005030000…

Dropdown(description='هفته:', layout=Layout(width='95%'), options=(('2026-02-07  تا  2026-02-13', datetime.dat…

Output()

In [29]:
branch = branches_list[0]  # یا اسم دقیق یک شعبه
code_goods = fc_daily[ITEM_COL].iloc[0]  # یا کد دقیق یک کالا

w = fc_weekly[(fc_weekly[BRANCH_COL] == branch) & (fc_weekly[ITEM_COL] == code_goods)].sort_values("week_start")
w.head()


,Receiving Warehouse Name,code goods,title,unit,week_start,week_total


In [30]:
!pip -q install ipywidgets plotly
from google.colab import output
output.enable_custom_widget_manager()

import pandas as pd
import numpy as np
import plotly.graph_objects as go
import ipywidgets as widgets
from IPython.display import display, clear_output

# اطمینان از datetime
fc_daily["date"] = pd.to_datetime(fc_daily["date"])
fc_weekly["week_start"] = pd.to_datetime(fc_weekly["week_start"])

BRANCH_COL = "Receiving Warehouse Name"
ITEM_COL = "code goods"

out = widgets.Output()

# Dropdown ها
branches_list = sorted(fc_weekly[BRANCH_COL].dropna().unique().tolist())
dd_branch = widgets.Dropdown(options=branches_list, description="شعبه:", layout=widgets.Layout(width="95%"))
dd_item   = widgets.Dropdown(options=[], description="کالا:", layout=widgets.Layout(width="95%"))
dd_week   = widgets.Dropdown(options=[], description="هفته:", layout=widgets.Layout(width="95%"))

def update_items(branch):
    subset = (fc_weekly[fc_weekly[BRANCH_COL] == branch][[ITEM_COL, "title", "unit"]]
              .drop_duplicates()
              .sort_values(["title", ITEM_COL]))
    opts = [(f"{r['title']} | کد:{r[ITEM_COL]} | واحد:{r['unit']}", r[ITEM_COL])
            for _, r in subset.iterrows()]
    dd_item.options = opts
    if opts:
        dd_item.value = opts[0][1]

def update_weeks(branch, code_goods):
    w = (fc_weekly[(fc_weekly[BRANCH_COL] == branch) & (fc_weekly[ITEM_COL] == code_goods)]
         .sort_values("week_start"))
    opts = []
    for ws in w["week_start"].unique():
        ws = pd.Timestamp(ws)
        we = ws + pd.Timedelta(days=6)
        opts.append((f"{ws.date()} تا {we.date()}", ws.date()))
    dd_week.options = opts
    if opts:
        dd_week.value = opts[0][1]

def render(branch, code_goods, week_start_date):
    with out:
        clear_output(wait=True)

        # ---------- داده‌ها ----------
        w = (fc_weekly[(fc_weekly[BRANCH_COL] == branch) & (fc_weekly[ITEM_COL] == code_goods)]
             .sort_values("week_start"))
        if w.empty:
            display("برای این انتخاب داده هفتگی وجود ندارد.")
            return

        title = w["title"].iloc[0]
        unit  = w["unit"].iloc[0]

        # ---------- نمودار هفتگی حرفه‌ای (Line + Area + Band) ----------
        w = w.copy()
        # Band نمایشی برای ارائه (بعداً می‌تونیم واقعی‌اش رو از بک‌تست بسازیم)
        w["p20"] = w["week_total"] * 0.85
        w["p80"] = w["week_total"] * 1.15

        fig_w = go.Figure()
        fig_w.add_trace(go.Scatter(
            x=w["week_start"], y=w["p20"],
            mode="lines", line=dict(width=0),
            showlegend=False, name="P20"
        ))
        fig_w.add_trace(go.Scatter(
            x=w["week_start"], y=w["p80"],
            mode="lines", fill="tonexty",
            name="Uncertainty band (P20–P80)",
            opacity=0.25
        ))
        fig_w.add_trace(go.Scatter(
            x=w["week_start"], y=w["week_total"],
            mode="lines+markers",
            name="Weekly forecast",
            line=dict(width=3)
        ))
        fig_w.update_layout(
            title=f"Weekly Forecast (Sat–Fri) - {branch} | {title} ({unit})",
            height=360,
            hovermode="x unified",
            xaxis_title="Week start (Saturday)",
            yaxis_title=f"Weekly total ({unit})"
        )
        fig_w.show()

        # ---------- جزئیات هفته انتخابی ----------
        ws = pd.Timestamp(week_start_date)
        we = ws + pd.Timedelta(days=6)

        ddf = fc_daily[(fc_daily[BRANCH_COL] == branch) & (fc_daily[ITEM_COL] == code_goods)].copy()
        week_df = ddf[(ddf["date"] >= ws) & (ddf["date"] <= we)].sort_values("date")

        if week_df.empty:
            display("برای این هفته داده روزانه وجود ندارد.")
            return

        week_sum = week_df["yhat"].sum()
        week_avg = week_df["yhat"].mean()
        week_max = week_df["yhat"].max()

        display(widgets.HTML(f"""
        <div style="font-family:Tahoma; line-height:1.9; margin:8px 0; padding:10px; border:1px solid #ddd; border-radius:10px;">
          <b>جزئیات هفته:</b> {ws.date()} تا {we.date()}<br/>
          <b>جمع هفته:</b> {week_sum:,.2f} {unit} &nbsp; | &nbsp;
          <b>میانگین روزانه:</b> {week_avg:,.2f} {unit} &nbsp; | &nbsp;
          <b>بیشترین روز:</b> {week_max:,.2f} {unit}
        </div>
        """))

        # ---------- نمودار روزانه حرفه‌ای (Line + markers + Holiday + Friday shading) ----------
        fig_d = go.Figure()
        fig_d.add_trace(go.Scatter(
            x=week_df["date"],
            y=week_df["yhat"],
            mode="lines+markers",
            name="Daily forecast",
            line=dict(width=3)
        ))

        hol = week_df[week_df["is_holiday"] == True]
        if not hol.empty:
            fig_d.add_trace(go.Scatter(
                x=hol["date"],
                y=hol["yhat"],
                mode="markers",
                name="Holiday",
                marker=dict(size=12, symbol="diamond")
            ))

        # هایلایت جمعه‌ها (dow_iran==6)
        fri = week_df[week_df["dow_iran"] == 6]
        for d in fri["date"]:
            fig_d.add_vrect(
                x0=d - pd.Timedelta(hours=12),
                x1=d + pd.Timedelta(hours=12),
                fillcolor="gray",
                opacity=0.12,
                line_width=0
            )

        fig_d.update_layout(
            title=f"Daily Detail (Selected Week) - {branch} | {title} ({unit})",
            height=380,
            hovermode="x unified",
            xaxis_title="Date",
            yaxis_title=f"Predicted daily demand ({unit})"
        )
        fig_d.show()

        # جدول شنبه تا جمعه
        display(week_df[[BRANCH_COL, ITEM_COL, "title", "unit", "date", "weekday_fa", "yhat", "is_holiday"]].reset_index(drop=True))

def on_branch(change):
    if change["name"] == "value":
        update_items(change["new"])
        if dd_item.value is not None:
            update_weeks(dd_branch.value, dd_item.value)
            render(dd_branch.value, dd_item.value, dd_week.value)

def on_item(change):
    if change["name"] == "value":
        update_weeks(dd_branch.value, change["new"])
        render(dd_branch.value, dd_item.value, dd_week.value)

def on_week(change):
    if change["name"] == "value":
        render(dd_branch.value, dd_item.value, change["new"])

dd_branch.observe(on_branch)
dd_item.observe(on_item)
dd_week.observe(on_week)

# init
update_items(dd_branch.value)
update_weeks(dd_branch.value, dd_item.value)
display(dd_branch, dd_item, dd_week, out)
render(dd_branch.value, dd_item.value, dd_week.value)


Dropdown(description='شعبه:', layout=Layout(width='95%'), options=('اسپئو رویال پالادیوم', 'برگر فکتوری ایران …

Dropdown(description='کالا:', layout=Layout(width='95%'), options=(('آب آلبالو زرشک 750 سی سی | کد:10005030000…

Dropdown(description='هفته:', layout=Layout(width='95%'), options=(('2026-02-07 تا 2026-02-13', datetime.date(…

Output()

In [31]:
import pandas as pd
import numpy as np

# اطمینان از datetime
fc_daily["date"] = pd.to_datetime(fc_daily["date"])

BRANCH_COL = "Receiving Warehouse Name"
ITEM_COL = "code goods"

# week_start (شنبه)
offset_to_saturday = (fc_daily["date"].dt.dayofweek - 5) % 7
fc_daily["week_start"] = fc_daily["date"] - pd.to_timedelta(offset_to_saturday, unit="D")

# Candlestick components from DAILY within each week
# Open: first day (Saturday), Close: last day (Friday), High/Low from daily, Avg
cs = (fc_daily.sort_values(["week_start","date"])
      .groupby([BRANCH_COL, ITEM_COL, "title", "unit", "week_start"], as_index=False)
      .agg(
          open=("yhat", "first"),
          close=("yhat", "last"),
          high=("yhat", "max"),
          low=("yhat", "min"),
          avg=("yhat", "mean"),
          week_total=("yhat","sum")
      ))

# Heatmap table: rows = week_start, columns = weekday_fa, values = yhat
# (برای هر شعبه-کالا جدا فیلتر می‌کنیم، پس اینجا فقط آماده‌سازی کلی)
dow_order = ["شنبه","یکشنبه","دوشنبه","سه‌شنبه","چهارشنبه","پنجشنبه","جمعه"]


In [32]:
!pip -q install ipywidgets plotly
from google.colab import output
output.enable_custom_widget_manager()

import plotly.graph_objects as go
import plotly.express as px
import ipywidgets as widgets
from IPython.display import display, clear_output

out = widgets.Output()

# Dropdown ها
branches_list = sorted(fc_daily[BRANCH_COL].dropna().unique().tolist())
dd_branch = widgets.Dropdown(options=branches_list, description="شعبه:", layout=widgets.Layout(width="95%"))
dd_item   = widgets.Dropdown(options=[], description="کالا:", layout=widgets.Layout(width="95%"))
dd_week   = widgets.Dropdown(options=[], description="هفته:", layout=widgets.Layout(width="95%"))

def update_items(branch):
    subset = (fc_daily[fc_daily[BRANCH_COL] == branch][[ITEM_COL, "title", "unit"]]
              .drop_duplicates()
              .sort_values(["title", ITEM_COL]))
    opts = [(f"{r['title']} | کد:{r[ITEM_COL]} | واحد:{r['unit']}", r[ITEM_COL])
            for _, r in subset.iterrows()]
    dd_item.options = opts
    if opts:
        dd_item.value = opts[0][1]

def update_weeks(branch, code_goods):
    w = cs[(cs[BRANCH_COL] == branch) & (cs[ITEM_COL] == code_goods)].sort_values("week_start")
    opts = []
    for ws in w["week_start"].unique():
        ws = pd.Timestamp(ws)
        we = ws + pd.Timedelta(days=6)
        opts.append((f"{ws.date()} تا {we.date()}", ws.date()))
    dd_week.options = opts
    if opts:
        dd_week.value = opts[0][1]

def render(branch, code_goods, week_start_date):
    with out:
        clear_output(wait=True)

        # ------------------- Filter base data -------------------
        ddf = fc_daily[(fc_daily[BRANCH_COL] == branch) & (fc_daily[ITEM_COL] == code_goods)].sort_values("date")
        if ddf.empty:
            display("برای این انتخاب داده‌ای وجود ندارد.")
            return

        title = ddf["title"].iloc[0]
        unit  = ddf["unit"].iloc[0]

        cs_df = cs[(cs[BRANCH_COL] == branch) & (cs[ITEM_COL] == code_goods)].sort_values("week_start")
        if cs_df.empty:
            display("برای این انتخاب داده Candlestick وجود ندارد.")
            return

        # ------------------- 1) Candlestick (هفتگی) -------------------
        fig_cs = go.Figure()

        fig_cs.add_trace(go.Candlestick(
            x=cs_df["week_start"],
            open=cs_df["open"],
            high=cs_df["high"],
            low=cs_df["low"],
            close=cs_df["close"],
            name="Weekly OHLC"
        ))

        # خط میانگین هفته
        fig_cs.add_trace(go.Scatter(
            x=cs_df["week_start"],
            y=cs_df["avg"],
            mode="lines+markers",
            name="Weekly avg",
            line=dict(width=2)
        ))

        fig_cs.update_layout(
            title=f"Candlestick Weekly (Sat open → Fri close) - {branch} | {title} ({unit})",
            height=420,
            xaxis_title="Week start (Saturday)",
            yaxis_title=f"Predicted daily level ({unit})",
            hovermode="x unified"
        )
        fig_cs.show()

        # ------------------- 2) Heatmap (هفته × روزهای هفته) -------------------
        # pivot values = yhat for each weekday within each week
        hm = ddf.pivot_table(index="week_start", columns="weekday_fa", values="yhat", aggfunc="sum").reset_index()
        # ترتیب ستون‌ها
        cols = ["week_start"] + [c for c in dow_order if c in hm.columns]
        hm = hm[cols].sort_values("week_start")

        # آماده‌سازی برای heatmap
        z = hm[[c for c in dow_order if c in hm.columns]].to_numpy()
        x = [c for c in dow_order if c in hm.columns]
        y = hm["week_start"].dt.date.astype(str).tolist()

        fig_hm = go.Figure(data=go.Heatmap(
            z=z,
            x=x,
            y=y,
            hoverongaps=False
        ))
        fig_hm.update_layout(
            title=f"Heatmap (Weeks × Weekdays) - {branch} | {title} ({unit})",
            height=420,
            xaxis_title="روزهای هفته",
            yaxis_title="هفته (شروع شنبه)"
        )
        fig_hm.show()

        # ------------------- 3) Daily detail for selected week -------------------
        ws = pd.Timestamp(week_start_date)
        we = ws + pd.Timedelta(days=6)
        week_df = ddf[(ddf["date"] >= ws) & (ddf["date"] <= we)].copy()

        if week_df.empty:
            display("برای این هفته داده روزانه وجود ندارد.")
            return

        # KPI هفته
        week_sum = week_df["yhat"].sum()
        week_avg = week_df["yhat"].mean()
        week_max = week_df["yhat"].max()

        display(widgets.HTML(f"""
        <div style="font-family:Tahoma; line-height:1.9; margin:8px 0; padding:10px; border:1px solid #ddd; border-radius:10px;">
          <b>هفته انتخاب‌شده:</b> {ws.date()} تا {we.date()}<br/>
          <b>جمع هفته:</b> {week_sum:,.2f} {unit} &nbsp; | &nbsp;
          <b>میانگین روزانه:</b> {week_avg:,.2f} {unit} &nbsp; | &nbsp;
          <b>بیشترین روز:</b> {week_max:,.2f} {unit}
        </div>
        """))

        fig_d = go.Figure()
        fig_d.add_trace(go.Scatter(
            x=week_df["date"],
            y=week_df["yhat"],
            mode="lines+markers",
            name="Daily forecast",
            line=dict(width=3)
        ))

        # علامت‌گذاری تعطیلی‌ها
        hol = week_df[week_df["is_holiday"] == True]
        if not hol.empty:
            fig_d.add_trace(go.Scatter(
                x=hol["date"],
                y=hol["yhat"],
                mode="markers",
                name="Holiday",
                marker=dict(size=12, symbol="diamond")
            ))

        # هایلایت جمعه‌ها
        fri = week_df[week_df["dow_iran"] == 6]
        for d in fri["date"]:
            fig_d.add_vrect(
                x0=d - pd.Timedelta(hours=12),
                x1=d + pd.Timedelta(hours=12),
                fillcolor="gray",
                opacity=0.12,
                line_width=0
            )

        fig_d.update_layout(
            title=f"Daily Detail (Selected Week) - {branch} | {title} ({unit})",
            height=380,
            hovermode="x unified",
            xaxis_title="Date",
            yaxis_title=f"Predicted daily demand ({unit})"
        )
        fig_d.show()

        display(week_df[[BRANCH_COL, ITEM_COL, "title","unit","date","weekday_fa","yhat","is_holiday"]].reset_index(drop=True))

def on_branch(change):
    if change["name"] == "value":
        update_items(change["new"])
        if dd_item.value is not None:
            update_weeks(dd_branch.value, dd_item.value)
            render(dd_branch.value, dd_item.value, dd_week.value)

def on_item(change):
    if change["name"] == "value":
        update_weeks(dd_branch.value, change["new"])
        render(dd_branch.value, dd_item.value, dd_week.value)

def on_week(change):
    if change["name"] == "value":
        render(dd_branch.value, dd_item.value, change["new"])

dd_branch.observe(on_branch)
dd_item.observe(on_item)
dd_week.observe(on_week)

# init
update_items(dd_branch.value)
update_weeks(dd_branch.value, dd_item.value)
display(dd_branch, dd_item, dd_week, out)
render(dd_branch.value, dd_item.value, dd_week.value)


Dropdown(description='شعبه:', layout=Layout(width='95%'), options=('اسپئو رویال پالادیوم', 'برگر فکتوری ایران …

Dropdown(description='کالا:', layout=Layout(width='95%'), options=(('آب آلبالو زرشک 750 سی سی | کد:10005030000…

Dropdown(description='هفته:', layout=Layout(width='95%'), options=(('2026-02-07 تا 2026-02-13', datetime.date(…

Output()

In [33]:
import pandas as pd
import jdatetime

BRANCH_COL = "Receiving Warehouse Name"
ITEM_COL = "code goods"

# اطمینان از datetime
fc_daily["date"] = pd.to_datetime(fc_daily["date"])

# ساخت تاریخ شمسی
def to_jalali_str(dt):
    j = jdatetime.date.fromgregorian(date=dt.date())
    return f"{j.year:04d}/{j.month:02d}/{j.day:02d}"

final_out = fc_daily.copy()
final_out["date_jalali"] = final_out["date"].apply(to_jalali_str)

# مرتب‌سازی و انتخاب ستون‌های خروجی نهایی
final_out = final_out[[
    BRANCH_COL,                 # واحد درخواست‌کننده
    ITEM_COL,                   # کد کالا
    "title",                    # نام کالا
    "unit",                     # واحد اندازه گیری کالا
    "date",                     # تاریخ میلادی
    "date_jalali",              # تاریخ شمسی
    "weekday_fa",               # روز هفته ایران
    "is_holiday",               # تعطیل؟
    "yhat"                      # مقدار پیش‌بینی
]].sort_values([BRANCH_COL, ITEM_COL, "date"]).reset_index(drop=True)

# ذخیره اکسل
out_path = "final_forecast_by_branch_item_daily_3months.xlsx"
with pd.ExcelWriter(out_path, engine="xlsxwriter") as w:
    final_out.to_excel(w, index=False, sheet_name="daily_forecast")

from google.colab import files
files.download(out_path)

final_out.head()


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

,Receiving Warehouse Name,code goods,title,unit,date,date_jalali,weekday_fa,is_holiday,yhat
0,اسپئو رویال پالادیوم,100010100002,سردست گوساله بدون استخوان,گرم,2026-02-10,1404/11/21,سه‌شنبه,False,272.905843
1,اسپئو رویال پالادیوم,100010100002,سردست گوساله بدون استخوان,گرم,2026-02-11,1404/11/22,چهارشنبه,True,1786.027807
2,اسپئو رویال پالادیوم,100010100002,سردست گوساله بدون استخوان,گرم,2026-02-12,1404/11/23,پنجشنبه,False,272.905843
3,اسپئو رویال پالادیوم,100010100002,سردست گوساله بدون استخوان,گرم,2026-02-13,1404/11/24,جمعه,False,272.905843
4,اسپئو رویال پالادیوم,100010100002,سردست گوساله بدون استخوان,گرم,2026-02-14,1404/11/25,شنبه,False,272.905843


In [34]:
# خلاصه هفتگی برای هر واحد + کالا
weekly_out = (fc_daily.groupby([BRANCH_COL, ITEM_COL, "title", "unit", "week_start"], as_index=False)
                     .agg(week_total=("yhat","sum"),
                          week_avg_daily=("yhat","mean"),
                          week_max_daily=("yhat","max")))

weekly_out["week_start"] = pd.to_datetime(weekly_out["week_start"])
weekly_out["week_start_jalali"] = weekly_out["week_start"].apply(to_jalali_str)

out_path = "final_forecast_branch_item_daily_and_weekly.xlsx"
with pd.ExcelWriter(out_path, engine="xlsxwriter") as w:
    final_out.to_excel(w, index=False, sheet_name="daily_forecast")
    weekly_out.to_excel(w, index=False, sheet_name="weekly_summary")

from google.colab import files
files.download(out_path)
weekly_out.head()


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

,Receiving Warehouse Name,code goods,title,unit,week_start,week_total,week_avg_daily,week_max_daily,week_start_jalali
0,اسپئو رویال پالادیوم,100010100002,سردست گوساله بدون استخوان,گرم,2026-02-07,2604.745336,651.186334,1786.027807,1404/11/18
1,اسپئو رویال پالادیوم,100010100002,سردست گوساله بدون استخوان,گرم,2026-02-14,1876.008006,268.001144,272.905843,1404/11/25
2,اسپئو رویال پالادیوم,100010100002,سردست گوساله بدون استخوان,گرم,2026-02-21,1431.437689,204.491098,238.572948,1404/12/02
3,اسپئو رویال پالادیوم,100010100002,سردست گوساله بدون استخوان,گرم,2026-02-28,1286.978176,183.854025,272.905843,1404/12/09
4,اسپئو رویال پالادیوم,100010100002,سردست گوساله بدون استخوان,گرم,2026-03-07,3423.462865,489.066124,1786.027807,1404/12/16


In [35]:
import pandas as pd
import numpy as np
import jdatetime
from google.colab import files

BRANCH_COL = "Receiving Warehouse Name"
ITEM_COL = "code goods"

fc_daily["date"] = pd.to_datetime(fc_daily["date"])

def to_jalali_str(dt):
    j = jdatetime.date.fromgregorian(date=dt.date())
    return f"{j.year:04d}/{j.month:02d}/{j.day:02d}"

# تعیین اینکه کدام unit باید عدد صحیح شود
INTEGER_UNITS_KEYWORDS = [
    "عدد","بطری","شیشه","قوطی","بسته","پک","کیس","کارتن","جعبه",
    "سطل","پاکت","رول","دست","پرس","لیوان"
]
DECIMAL_UNITS_KEYWORDS = [
    "گرم","کیلو","کیلوگرم","kg","g","لیتر","l","ml","میلی","cc"
]

def smart_round(y, unit):
    if unit is None:
        return round(float(y), 2)
    u = str(unit).strip().lower()

    # اگر شبیه واحدهای تعدادی بود → عدد صحیح
    for k in INTEGER_UNITS_KEYWORDS:
        if k in u:
            return int(np.round(float(y)))

    # اگر شبیه واحدهای وزنی/حجمی بود → دو رقم اعشار
    for k in DECIMAL_UNITS_KEYWORDS:
        if k in u:
            return round(float(y), 2)

    # پیش‌فرض: دو رقم اعشار
    return round(float(y), 2)

final_out = fc_daily.copy()
final_out["date_jalali"] = final_out["date"].apply(to_jalali_str)

# رُندینگ
final_out["yhat_rounded"] = final_out.apply(lambda r: smart_round(r["yhat"], r["unit"]), axis=1)

# خروجی نهایی
final_out = final_out[[
    BRANCH_COL,
    ITEM_COL,
    "title",
    "unit",
    "date",
    "date_jalali",
    "weekday_fa",
    "is_holiday",
    "yhat",
    "yhat_rounded"
]].sort_values([BRANCH_COL, ITEM_COL, "date"]).reset_index(drop=True)

out_path = "final_forecast_by_branch_item_daily_3months_rounded.xlsx"
with pd.ExcelWriter(out_path, engine="xlsxwriter") as w:
    final_out.to_excel(w, index=False, sheet_name="daily_forecast")

files.download(out_path)

final_out.head(20)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

,Receiving Warehouse Name,code goods,title,unit,date,date_jalali,weekday_fa,is_holiday,yhat,yhat_rounded
0,اسپئو رویال پالادیوم,100010100002,سردست گوساله بدون استخوان,گرم,2026-02-10,1404/11/21,سه‌شنبه,False,272.905843,272.91
1,اسپئو رویال پالادیوم,100010100002,سردست گوساله بدون استخوان,گرم,2026-02-11,1404/11/22,چهارشنبه,True,1786.027807,1786.03
2,اسپئو رویال پالادیوم,100010100002,سردست گوساله بدون استخوان,گرم,2026-02-12,1404/11/23,پنجشنبه,False,272.905843,272.91
3,اسپئو رویال پالادیوم,100010100002,سردست گوساله بدون استخوان,گرم,2026-02-13,1404/11/24,جمعه,False,272.905843,272.91
4,اسپئو رویال پالادیوم,100010100002,سردست گوساله بدون استخوان,گرم,2026-02-14,1404/11/25,شنبه,False,272.905843,272.91
5,اسپئو رویال پالادیوم,100010100002,سردست گوساله بدون استخوان,گرم,2026-02-15,1404/11/26,یکشنبه,False,272.905843,272.91
6,اسپئو رویال پالادیوم,100010100002,سردست گوساله بدون استخوان,گرم,2026-02-16,1404/11/27,دوشنبه,False,272.905843,272.91
7,اسپئو رویال پالادیوم,100010100002,سردست گوساله بدون استخوان,گرم,2026-02-17,1404/11/28,سه‌شنبه,False,272.905843,272.91
8,اسپئو رویال پالادیوم,100010100002,سردست گوساله بدون استخوان,گرم,2026-02-18,1404/11/29,چهارشنبه,False,272.905843,272.91
9,اسپئو رویال پالادیوم,100010100002,سردست گوساله بدون استخوان,گرم,2026-02-19,1404/11/30,پنجشنبه,False,272.905843,272.91


In [38]:
# =========================
# 0) Install (Colab)
# =========================
!pip -q install pandas numpy lightgbm openpyxl

import pandas as pd
import numpy as np
import math
from lightgbm import LGBMRegressor

# =========================
# 1) Config
# =========================
PATH = "/content/drive/MyDrive/data/پیش بینی -تولید.xlsx"  # مسیر فایل اکسل
BRANCH_COL = "Receiving Warehouse Name"
ITEM_COL   = "code goods"
TITLE_COL  = "Tittle Goods"          # ✅ اسم کالا (اگر فرق داشت همینجا عوض کن)
VAL_COL    = "Value (in base unit)"
DATE_COL   = "Date"

TEST_DAYS = 28        # بک‌تست: 28 روز آخر هر سری
MIN_TRAIN_DAYS = 60   # حداقل طول آموزش برای نگه داشتن سری

# =========================
# 2) Load
# =========================
df = pd.read_excel(PATH)

# تاریخ مثل 25/02/09  -> 2025-02-09
df["date"] = pd.to_datetime("20" + df[DATE_COL].astype(str), errors="coerce")
df = df.dropna(subset=["date"])

# =========================
# 3) Build code->title map (robust)
#    اگر یک کد کالا چند عنوان داشته باشد، پرتکرارترین را برمی‌داریم
# =========================
title_map = (
    df[[ITEM_COL, TITLE_COL]]
    .dropna()
    .astype({ITEM_COL: str, TITLE_COL: str})
    .groupby([ITEM_COL, TITLE_COL])
    .size()
    .reset_index(name="cnt")
    .sort_values(["cnt"], ascending=False)
    .drop_duplicates(subset=[ITEM_COL])
    [[ITEM_COL, TITLE_COL]]
)

# =========================
# 4) Aggregate to daily y
# =========================
g = (df.groupby([BRANCH_COL, ITEM_COL, "date"], as_index=False)[VAL_COL]
       .sum()
       .rename(columns={VAL_COL: "y"}))

# =========================
# 5) Full daily panel (fill missing days with 0) - vectorized
# =========================
daily = (g.set_index("date")
           .groupby([BRANCH_COL, ITEM_COL])["y"]
           .resample("D")
           .sum()
           .reset_index())
daily["y"] = daily["y"].fillna(0.0)

daily["series_id"] = daily[BRANCH_COL].astype(str) + "||" + daily[ITEM_COL].astype(str)
daily = daily.sort_values(["series_id", "date"]).reset_index(drop=True)

# =========================
# 6) Features (simple + lags + rolling)
# =========================
daily["dow"] = daily["date"].dt.dayofweek
daily["dom"] = daily["date"].dt.day

for lag in (1, 7, 14):
    daily[f"lag{lag}"] = daily.groupby("series_id")["y"].shift(lag)

daily["roll7"]  = daily.groupby("series_id")["y"].transform(lambda s: s.shift(1).rolling(7).mean())
daily["roll28"] = daily.groupby("series_id")["y"].transform(lambda s: s.shift(1).rolling(28).mean())

daily = daily.dropna(subset=["lag1","lag7","lag14","roll7","roll28"]).reset_index(drop=True)

# =========================
# 7) Temporal split: last TEST_DAYS per series = test
# =========================
n   = daily.groupby("series_id")["y"].transform("size")
pos = daily.groupby("series_id").cumcount()
is_test = pos >= (n - TEST_DAYS)

train_df = daily[~is_test].copy()
test_df  = daily[is_test].copy()

# keep series with enough training history
ok = train_df.groupby("series_id").size()
ok_series = ok[ok >= MIN_TRAIN_DAYS].index
train_df = train_df[train_df["series_id"].isin(ok_series)]
test_df  = test_df[test_df["series_id"].isin(ok_series)]

features = ["dow","dom","lag1","lag7","lag14","roll7","roll28"]

# =========================
# 8) Train model
# =========================
model = LGBMRegressor(
    n_estimators=200,
    learning_rate=0.08,
    num_leaves=31,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    n_jobs=-1,
    force_row_wise=True
)
model.fit(train_df[features], train_df["y"])

# =========================
# 9) Predict + metrics
# =========================
test_df = test_df.copy()
test_df["yhat"] = model.predict(test_df[features])

test_df["ae"]  = (test_df["y"] - test_df["yhat"]).abs()
test_df["se"]  = (test_df["y"] - test_df["yhat"])**2
test_df["ape"] = np.where(test_df["y"] > 0, test_df["ae"]/test_df["y"], np.nan)

metrics = (test_df.groupby([BRANCH_COL, ITEM_COL], as_index=False)
           .agg(
               MAE=("ae","mean"),
               RMSE=("se", lambda x: math.sqrt(np.mean(x))),
               MAPE=("ape", lambda x: np.nanmean(x)*100),
               n_test_points=("y","count"),
           ))

# =========================
# 10) Add title next to code goods ✅
# =========================
metrics[ITEM_COL] = metrics[ITEM_COL].astype(str)
metrics = metrics.merge(title_map, on=ITEM_COL, how="left")

# reorder columns
metrics = metrics[[BRANCH_COL, ITEM_COL, TITLE_COL, "MAE", "RMSE", "MAPE", "n_test_points"]] \
                 .sort_values(["MAPE","MAE"], ascending=[True, True]) \
                 .reset_index(drop=True)

# overall
overall = pd.DataFrame([{
    "MAE_overall":  test_df["ae"].mean(),
    "RMSE_overall": math.sqrt(test_df["se"].mean()),
    "MAPE_overall": np.nanmean(test_df["ape"])*100,
    "n_test_points": len(test_df),
    "n_series": metrics.shape[0]
}])

# =========================
# 11) Save outputs
# =========================
metrics.to_csv("/content/metrics_by_branch_item.csv", index=False, encoding="utf-8-sig")
overall.to_csv("/content/metrics_overall.csv", index=False, encoding="utf-8-sig")

print("✅ Saved: /content/metrics_by_branch_item.csv")
print("✅ Saved: /content/metrics_overall.csv")
display(overall)
display(metrics.head(10))


[LightGBM] [Info] Total Bins 1261
[LightGBM] [Info] Number of data points in the train set: 193748, number of used features: 7
[LightGBM] [Info] Start training from score 2829.345612
✅ Saved: /content/metrics_by_branch_item.csv
✅ Saved: /content/metrics_overall.csv


,MAE_overall,RMSE_overall,MAPE_overall,n_test_points,n_series
0,4789.144406,42220.132627,1488.020105,35728,1276


,Receiving Warehouse Name,code goods,Tittle Goods,MAE,RMSE,MAPE,n_test_points
0,اسپئو رویال پالادیوم,100050500003,چای ترش دکتر بین 75 گرمی,344.187160,353.089942,8.877161,28
1,برگر فکتوری ایران مال,300030200057,بشقاب میوه خوری چهار گوش مشکی,335.635632,349.763455,9.799080,28
2,اسپئو رویال پالادیوم,300010800057,کاسه 310 سی سی طب پلاستیک,334.415358,348.379231,12.072529,28
3,اسپئو رویال پالادیوم,100050500005,دمنوش گیاهی دکتر بین,344.722875,353.307098,12.381885,28
4,اسپئو رویال پالادیوم,300010700007,چنگال یکبار مصرف طرح گلدن,344.487921,353.277171,13.903549,28
5,اسپئو رویال پالادیوم,300010700014,قاشق یکبار مصرف طرح گلدن,344.487921,353.277171,13.903549,28
6,اسپئو رویال پالادیوم,300010800029,ظرف 703 اچ دی,345.080018,353.360962,14.572338,28
7,چیکن فکتوری بام لند,100070100024,کلم بروکلی,321.423579,341.503383,15.692103,28
8,روماتای پالادیوم,100100500030,پودر عصاره مرغ چینی 500گرمی,330.007671,340.499843,16.046196,28
9,چیکن فکتوری ایران مال,100070100017,کاهو میکس سالادی,330.007671,340.499843,16.046196,28
